In [36]:
# Target channels to monitor
# These are entertainment/movie commentary accounts that frequently post film clips.

handles = ["@ScreenRantPlus", "@CinemaSins", "@DylanIsInTrouble", "@MovieClips"]

In [37]:
# Import required libraries
import os
import requests
import pandas as pd
import yt_dlp
from datetime import datetime

# YouTube Data API key
API_KEY = PASTE_KEY

# Define storage folders. 
# Output : stores data and reports
# Preserved : stores downloaded video evidence
output_directory = "output"
preserved_directory = "preserved"

os.makedirs(output_directory, exist_ok="True")
os.makedirs(preserved_directory, exist_ok="True")

print("Folders ready.")

Folders ready.


In [38]:
# Step 1: Convert channel handles into channel IDs
# The YouTube API requires a channel ID to retrieve videos

def get_channel_id(handle):
    url = "https://www.googleapis.com/youtube/v3/search"

    params = {
        "part": "snippet",
        "q": handle,
        "type": "channel",
        "maxResults": 1,
        "key": API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()

    return data["items"][0]["snippet"]["channelId"]

In [39]:
# Step 2: Get recent videos from a channel
def get_recent_videos(channel_id):
    url = "https://www.googleapis.com/youtube/v3/search"

    params = {
        "part": "snippet",
        "channelId": channel_id,
        "type": "video",
        "order": "date",
        "maxResults": 10,
        "key": API_KEY
    }

    response = requests.get(url, params=params)
    items = response.json().get("items", [])

    videos = []

    for item in items:
        video_id = item["id"]["videoId"]
        title = item["snippet"]["title"]
        published = item["snippet"]["publishedAt"]
        channel_title = item["snippet"]["channelTitle"]

        video_url = f"https://www.youtube.com/watch?v={video_id}"

        videos.append({
            "video_id": video_id,
            "title": title,
            "published_at": published,
            "channel_title": channel_title,
            "video_url": video_url
        })
    return videos

In [40]:
cid = get_channel_id("@ScreenRantPlus")
videos = get_recent_videos(cid)

print(videos[0])

{'video_id': 'W7kHXgcwacQ', 'title': 'Psycho Killer Shot Two Very Different Versions', 'published_at': '2026-02-26T01:00:40Z', 'channel_title': 'Screen Rant Plus', 'video_url': 'https://www.youtube.com/watch?v=W7kHXgcwacQ'}


In [41]:
all_videos = []

for handle in handles:
    cid = get_channel_id(handle)
    vids = get_recent_videos(cid)

    for v in vids:
        v["handle"] = handle
        all_videos.append(v)

df = pd.DataFrame(all_videos)
df.head()

,video_id,title,published_at,channel_title,video_url,handle
0,W7kHXgcwacQ,Psycho Killer Shot Two Very Different Versions,2026-02-26T01:00:40Z,Screen Rant Plus,https://www.youtube.com/watch?v=W7kHXgcwacQ,@ScreenRantPlus
1,lafLbYrYXO0,Egg &amp; Dunk&#39;s Growth Will Be Reflected ...,2026-02-26T00:00:53Z,Screen Rant Plus,https://www.youtube.com/watch?v=lafLbYrYXO0,@ScreenRantPlus
2,Q0rxyKpYdUY,The Last Thing He Told Me Cast Debate Romantic...,2026-02-25T23:00:28Z,Screen Rant Plus,https://www.youtube.com/watch?v=Q0rxyKpYdUY,@ScreenRantPlus
3,ANkCNR8PikE,Monarch: Legacy of Monsters&#39;s Mari Yamamot...,2026-02-25T16:00:13Z,Screen Rant Plus,https://www.youtube.com/watch?v=ANkCNR8PikE,@ScreenRantPlus
4,_uMplYyLJqs,"Monarch: Legacy of Monsters Stars Anna Sawai, ...",2026-02-25T14:16:43Z,Screen Rant Plus,https://www.youtube.com/watch?v=_uMplYyLJqs,@ScreenRantPlus


In [42]:
# Step 3: Preserve video evidence

def download_one(url):
    ydl_opts = {
        'outtmpl': 'preserved/%(id)s.%(ext)s'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

In [43]:
for url in df["video_url"]:
    try:
        download_one(url)
    except Exception as e:
        print("Failed:", url)

[youtube] Extracting URL: https://www.youtube.com/watch?v=W7kHXgcwacQ
[youtube] W7kHXgcwacQ: Downloading webpage


[youtube] W7kHXgcwacQ: Downloading android vr player API JSON


[info] W7kHXgcwacQ: Downloading 1 format(s): 18
[download] Destination: preserved\W7kHXgcwacQ.mp4
[download] 100% of    2.28MiB in 00:00:00 at 5.84MiB/s   
[youtube] Extracting URL: https://www.youtube.com/watch?v=lafLbYrYXO0
[youtube] lafLbYrYXO0: Downloading webpage


[youtube] lafLbYrYXO0: Downloading android vr player API JSON


[info] lafLbYrYXO0: Downloading 1 format(s): 18
[download] preserved\lafLbYrYXO0.mp4 has already been downloaded
[download] 100% of    4.32MiB
[youtube] Extracting URL: https://www.youtube.com/watch?v=Q0rxyKpYdUY
[youtube] Q0rxyKpYdUY: Downloading webpage


...

[info] DU6jTEniMKk: Downloading 1 format(s): 18
[download] preserved\DU6jTEniMKk.mp4 has already been downloaded
[download] 100% of    6.42MiB
[youtube] Extracting URL: https://www.youtube.com/watch?v=Chp-a9TULaw
[youtube] Chp-a9TULaw: Downloading webpage


[youtube] Chp-a9TULaw: Downloading android vr player API JSON


[info] Chp-a9TULaw: Downloading 1 format(s): 18
[download] preserved\Chp-a9TULaw.mp4 has already been downloaded
[download] 100% of    2.02MiB
[youtube] Extracting URL: https://www.youtube.com/watch?v=01nzCEriFAA
[youtube] 01nzCEriFAA: Downloading webpage


[youtube] 01nzCEriFAA: Downloading android vr player API JSON


[info] 01nzCEriFAA: Downloading 1 format(s): 18
[download] preserved\01nzCEriFAA.mp4 has already been downloaded
[download] 100% of   16.87MiB


In [45]:
# Step 4: Build the Excel report

df["preserved_filename"] = df["video_id"].astype(str) + ".mp4"
df["preserved_path"] = "preserved/" + df["preserved_filename"]

df.head()

,video_id,title,published_at,channel_title,video_url,handle,preserved_filename,preserved_path
0,W7kHXgcwacQ,Psycho Killer Shot Two Very Different Versions,2026-02-26T01:00:40Z,Screen Rant Plus,https://www.youtube.com/watch?v=W7kHXgcwacQ,@ScreenRantPlus,W7kHXgcwacQ.mp4,preserved/W7kHXgcwacQ.mp4
1,lafLbYrYXO0,Egg &amp; Dunk&#39;s Growth Will Be Reflected ...,2026-02-26T00:00:53Z,Screen Rant Plus,https://www.youtube.com/watch?v=lafLbYrYXO0,@ScreenRantPlus,lafLbYrYXO0.mp4,preserved/lafLbYrYXO0.mp4
2,Q0rxyKpYdUY,The Last Thing He Told Me Cast Debate Romantic...,2026-02-25T23:00:28Z,Screen Rant Plus,https://www.youtube.com/watch?v=Q0rxyKpYdUY,@ScreenRantPlus,Q0rxyKpYdUY.mp4,preserved/Q0rxyKpYdUY.mp4
3,ANkCNR8PikE,Monarch: Legacy of Monsters&#39;s Mari Yamamot...,2026-02-25T16:00:13Z,Screen Rant Plus,https://www.youtube.com/watch?v=ANkCNR8PikE,@ScreenRantPlus,ANkCNR8PikE.mp4,preserved/ANkCNR8PikE.mp4
4,_uMplYyLJqs,"Monarch: Legacy of Monsters Stars Anna Sawai, ...",2026-02-25T14:16:43Z,Screen Rant Plus,https://www.youtube.com/watch?v=_uMplYyLJqs,@ScreenRantPlus,_uMplYyLJqs.mp4,preserved/_uMplYyLJqs.mp4


In [46]:
report_path = "output/movie_clip_copyright.xlsx"
df.to_excel(report_path, index=False)
print("Saved report to:", report_path)

Saved report to: output/movie_clip_copyright.xlsx
